In [ ]:
from pathlib import Path


fname = Path("jan_2021.nc4").absolute()

In [ ]:
import cf_xarray
import xarray as xr


ds = xr.open_dataset(fname, drop_variables="time_bounds")
ds

In [ ]:
variables = [
#     "grid_longitude",
#     "grid_latitude",
#     "k",
#     "grid_depth",
    "depth",
#     "zactive",
#     "area",
    "waterlevel",
    "velocity_x",
    "velocity_y",
#     "velocity_omega",
#     "velocity_z",
#     "tau_x",
#     "tau_y",
#     "density",
#     "salinity",
#     "temperature",
#     "tke",
#     "eps",
#     "viscosity_z",
#     "diffusivity_z",
#     "Ri",
]

subset = ds[variables].sel(
    m=slice(-49, -47.5)  # Cut some of the western part of the domain
).isel(
    {"Layer": 0}  # just the surface
)

In [ ]:
import humanize


print(f"Tamanho dos dados originais {humanize.naturalsize(ds.nbytes)=}")
print(f"Apenas com as variáveis necessárias {humanize.naturalsize(subset.nbytes)=}")

In [ ]:
print(f"Start: {ds.cf['T'][0].to_numpy()}\nEnd: {ds.cf['T'][-1].to_numpy()}")

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


def creat_map(projection=ccrs.PlateCarree(), figsize=(9, 9), bbox=None):
    fig, ax = plt.subplots(
        figsize=figsize,
        subplot_kw={
            "projection": projection,
        }
    )
    if bbox:
        ax.set_extent(bbox)
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.plot(-48.66, -26.89, "ro", label="Itajaí")
    ax.coastlines(resolution="10m")
    return fig, ax

fig, ax = creat_map()
subset.cf[["sea_floor_depth_below_sea_level"]]["depth"].T.plot.contourf(ax=ax);

Let's check the first layer of the first day.

We need to convert the `ocean_sigma_coordinate` coordinate there to z later!

In [ ]:
import numpy as np


def normalize_speed(u, v):
    u_norm = u / np.sqrt(u ** 2.0 + v ** 2.0)
    v_norm = v / np.sqrt(u ** 2.0 + v ** 2.0)
    speed = (u ** 2 + v ** 2) ** 0.5
    return (u_norm, v_norm, speed)

In [ ]:
first_tstep = subset.sel({"time": "2021-01-01T00:00:00"})

In [ ]:
bbox = first_tstep["m"].min(), first_tstep["m"].max(), first_tstep["n"].min(), first_tstep["n"].max()
bbox = [v.to_numpy() for v in bbox]


u = first_tstep.cf["eastward_sea_water_velocity"].squeeze()
v = first_tstep.cf["northward_sea_water_velocity"].squeeze()

u_norm, v_norm, speed = normalize_speed(u, v)

fig, ax = creat_map(bbox=bbox)
ax.contourf(first_tstep["longitude"], first_tstep["latitude"], speed)
ax.quiver(first_tstep["longitude"], first_tstep["latitude"], u_norm, v_norm, color="white", scale=50);

In [ ]:
def creat_map(projection=ccrs.PlateCarree(), figsize=(9, 9), bbox=None):
    fig, ax = plt.subplots(
        figsize=figsize,
        subplot_kw={
            "projection": projection,
        }
    )
    if bbox:
        ax.set_extent(bbox)
    ax.coastlines(resolution="10m")
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
def convert_npdatetime2str(npdatetime):
    import pandas as pd
    ts = pd.to_datetime(str(npdatetime))
    return ts.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
fig, ax = creat_map(bbox=bbox, figsize=(9, 9));

In [ ]:
from xmovie import Movie


vmin, vmax =-0.5, 0.5

def custom_plot_function(ds, fig, tt, *args, **kwargs):
    ax = fig.gca()
    vmin, vmax =-1.5, 1.9  # Data min/max but we will less to highligh the tides inside the bays.
    ds.isel(time=tt).plot(ax=ax, vmin=vmin, vmax=vmax, cmap="RdBu_r")

    npdatetime = ds["time"][tt].to_numpy()
    title = convert_npdatetime2str(npdatetime)
    ax.set_title(title)
    return None, None


chunked = subset.cf["sea_surface_height"].T.chunk({"time": 1})
mov = Movie(chunked, custom_plot_function, vmin=vmin, vmax=vmax)

## Check first time steps

In [ ]:
%matplotlib inline

In [ ]:
mov.preview(0)  # first hour

In [ ]:
mov.preview(1)  # second hour

In [ ]:
mov.preview(2)  # third hour

In [ ]:
mov.preview(11)  # twelfth hour

## Render movie

In [ ]:
filename = f"{fname.stem}.mp4"


mov.save(
    filename=filename,
    progress=True,
    parallel=True,
    parallel_compute_kwargs={"scheduler": "processes", "num_workers": 3},
    overwrite_existing=True,
)

In [ ]:
from IPython.display import HTML


HTML(f"""
    <video alt="test" controls style="outline:none; width:70%; height:70%;">
        <source src="{filename}" type="video/mp4">
    </video>
""")